In [ ]:
plt.rcParams["figure.figsize"]=30,15

In [ ]:
def randn_skew_fast(N, alpha=0.0, mean=0.0, var=1.0, seed = 123):
    sigma = alpha / np.sqrt(1.0 + alpha**2) 
    np.random.seed(seed)
    u0 = np.random.randn(N)
    v = np.random.randn(N)
    u1 = (sigma*u0 + np.sqrt(1.0 - sigma**2)*v) * var
    u1[u0 < 0] *= -1
    u1 = u1 + mean
    return u1

In [ ]:
def gen_paths(S0, r, sigma, T, M, I):
    dt = float(T) / M
    paths = np.zeros((M + 1, I), np.float64)
    paths[0] = S0
    for t in range(1, M + 1):
        rand = np.random.standard_normal(I)
        paths[t] = paths[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt +
                                         sigma * np.sqrt(dt) * rand)
    return paths

In [ ]:
N = 1000
alpha = -2
mean=25000
var = 8000

In [ ]:
median = mean - (alpha*var)/3
params = [alpha, mean, var]
plt.subplots(figsize=(12,4))
p = randn_skew_fast(N, alpha, median, var)
#sns.distplot(p)
p = p.astype('int', copy=False)
sns.kdeplot(data=abs(p))

In [ ]:
r = 0.035
sigma = 0.1
T = 1
deltat = T / N
i = 365*2

In [ ]:
np.random.seed(123)
S0 = 1.22
paths_eur = gen_paths(S0, r, sigma, T, i, N)
np.random.seed(133)
S0 = 1.7
paths_gbp = gen_paths(S0, r, sigma, T, i, N)

In [ ]:
plt.plot(paths_eur)
plt.show()

In [ ]:
plt.plot(paths_gbp)
plt.show()

In [ ]:
df_eur = pd.DataFrame(paths_eur[:,-1])
df_gbp = pd.DataFrame(paths_gbp[:,-1])

In [ ]:
df_volume = pd.DataFrame(p)

## save to xlsx file

filepath = 'simulation_data.xlsx'

df_volume.to_excel(filepath, sheet_name='volume_risk', index=False)

In [ ]:
with pd.ExcelWriter(filepath, engine='openpyxl') as writer:
    writer.book = load_workbook(filepath)
    df_eur.to_excel(writer, sheet_name='fx_risk_eur')
    df_gbp.to_excel(writer, sheet_name='fx_risk_gbp')
    